# Word Embeddings : le modèle Word2Vec

## Imports

In [2]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [3]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [4]:
infile = f"../data/sents_Netto.txt"
sentences = MySentences(infile)


### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [5]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [6]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [7]:
len(bigram_phrases.vocab.keys())

2487856

Prenons une clé au hasard :

In [8]:
key_ = list(bigram_phrases.vocab.keys())[79]
print(key_)

'##


Le dictionnaire indique le score de cette coocurrence :

In [9]:
bigram_phrases.vocab[key_]

40

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [10]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [11]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [12]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

In [13]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

#trigram_phrases = Phrases(bigram_phraser[sentences])

#trigram_phraser = Phraser(phrases_model=trigram_phrases)

corpus = list(bigram_phraser[sentences])

### Création d'un corpus d'unigrams, bigrams, trigrams

In [14]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [15]:
print(corpus[:100])

[['i', '*', 'PS', 'n', 'belgique', 'et', 'a', 'paris', '84', '-', 'annee', 'vendredi', '26', 'septembre', '1m3', 'administration_et_redaction', '1', 'i', '>>', 'ruo', '<<', 'i', '<<', 'mi', 'sarblc', '>>>>>>', 'bruxellt', '-', 's', '*>>', 'bureaux', 'parisiels', ':', '11', ',', 'place', 'de', 'la_bourse_abonnements', ':', "'##", 'edition_quotidienne_belgique', '.'], ['ub', '>>,', '20', 'fr', '.'], ['e', 'mois', '.'], ['iflfr', '.'], ['3_mois', ',', '5', 'fr', '.'], ['lukubdubfilsr', '.-_dj', '28', 'ir', '>>', '15', 'fr', '.'], ['>>', '8', 'fr', '.'], ['efuigeb', '>', '40', 'fr', '.'], ['22', 'fr', '.'], ['>>', '12', 'ir', '.'], ['edition_hebdomadaire', "'", 'it', '-', 'lirmtionaii', 'et', 'd', "'", 'oiiln', '-', 'merl', '10_pages', ',', 'paraissant_le', 'mercredi', 'un', 'tan', ',...', 'v', 'rvii', 'fran', '<>>', 'six', 'mol', '>>', 'ii', ';', 'franc', '->>', 'k', '>>', 'nom', 'vendredi', '26', 'septembre_1913', '<<', 'mois', 'tfllltifws', 'eas', 'joeb', '.', "'"], ['cossebvahok', 'sas

## Entrainement d'un modèle Word2Vec sur ce corpus

In [24]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=6, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=6, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: user 59.3 s, sys: 391 ms, total: 59.7 s
Wall time: 19.6 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [25]:
outfile = f"../data/newspapers6.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [26]:
model = Word2Vec.load("../data/newspapers6.model")

### Imprimer le vecteur d'un terme

In [27]:
model.wv["ministre"]

array([ 3.8294415 ,  0.62090427,  1.6034708 ,  0.88809747,  0.06035669,
        2.7277308 ,  0.41118413,  4.0942874 ,  2.075397  ,  0.96273476,
        1.3528792 ,  1.2451166 ,  4.2266593 ,  3.4429822 ,  1.0652195 ,
        1.7644215 , -0.5609715 ,  1.6637772 ,  1.0910776 , -1.5494561 ,
       -2.814812  ,  2.6933188 ,  3.9782495 , -2.1170995 , -1.0064204 ,
       -4.2632833 , -2.3043602 , -0.73400724,  1.9166998 ,  1.0718153 ,
        0.9441602 , -0.5960406 ], dtype=float32)

### Calculer la similarité entre deux termes

In [28]:
model.wv.similarity("colonisation", "souverain")

0.61044705

### Chercher les mots les plus proches d'un terme donné

In [22]:
model.wv.most_similar("droit", topn=10)

[('projet', 0.876010537147522),
 ('procede', 0.8609380722045898),
 ('vote', 0.8478446006774902),
 ('controle', 0.8478368520736694),
 ('contrat', 0.8400172591209412),
 ('pris', 0.8391669392585754),
 ('permis', 0.8312152624130249),
 ('traitement', 0.8246790766716003),
 ('concours', 0.8234037756919861),
 ('conseil', 0.8213037848472595)]

### Faire des recherches complexes à travers l'espace vectoriel

In [23]:
print(model.wv.most_similar(positive=['paris', 'londres','allemagne'], negative=['france', 'grande_bretagne'], topn=1))

[('berlin', 0.7091528177261353)]
